# (summary)Part-3_CNN

## Lab10-CNN

### 1. convolution

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.datasets as dsets # mnist dataset
import torchvision.transforms as transforms

# device, seed
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(7)
torch.manual_seed(7)
if device == 'cuda':
    torch.cuda.manual_seed_all(7)

mnist_train = dsets.MNIST(root='../Data/MNIST_data/',
                            train = True,
                            transform=transforms.ToTensor(),
                            download=False)
mnist_test = dsets.MNIST(root='../Data/MNIST_data/',
                            train = False,
                            transform=transforms.ToTensor(),
                            download=False)

# light model because of training without gpu
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 5, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(5, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(7*7*10, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# train
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                            batch_size = 20,
                                            shuffle = True,
                                            drop_last = True)

model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss().to(device)
nb_epochs = 100

total_batch = len(data_loader)
for epoch in range(nb_epochs):
    avg_cost = 0
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(x)

        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

# evaluation
with torch.no_grad():
     x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
     y_test = mnist_test.test_labels.to(device)

     prediction = model(x_test)
     correct_prediction = torch.argmax(prediction, 1) == y_test
     accuracy = correct_prediction.float().mean()
     print(f'Accuracy = {accuracy}')

### 2. custom image folder

### 3. advance_CNN